# Regressão Linear Multivariada (Adaline)

## Adaline (Adaptive Linear Neuron)

O Adaline é um modelo de neurônio oriundo da evolução do Perceptron (criado em 1957 por Frank Rosenblatt). Este modelo neural se diferencia por receber $N$ valores reais e retornar uma saída real.

$$f: \mathbb{R}^n \rightarrow \mathbb{R}$$

## Problema a ser resolvido

O Titanic foi um navio que afundou na madrugada de 14 de abril de 1912 durante sua viagem inaugural. O neurônio Adaline irá analisar o arquivo `titanic.csv` que possui as seguintes informações:

$$nome = x \in \{1, 2, 3\}$$

$$idade = x \in \mathbb{R} \mid x \ge 0$$

$$sexo = x \in \{0,1\}$$

$$sobreviveu = x \in \{0,1\}$$

Baseados nos valores de $nome$, $idade$ e $sexo$, determinar se determinada pessoa com essas características sobreviveu ou não.

In [ ]:
%pip install numpy pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display


## Função Geradora de Dados - $G(x)$ ou $y$

$$G(x) = ??$$

### Explicação

A função $G(x)$ representa o resultado eventos que já aconteceram na vida real ocasionados por determinados dados de entrada. Essa função ela é inacessível a nós, pois o evento já ocorreu e, para além disso, múltiplas variáveis são relacionadas ao evento ocorrido, variáveis essas que também são inacessíveis com total precisão.

Por exemplo: O Titanic, o maior navio de sua época, afundou no dia 15 de abril de 1912. Diversas variáveis determinaram naquela madrugada quem sobreviveu ao evento ou quem não sobreviveu. Acontece que o resultado desse evento ($G(x)$) está no passado e apenas quem esteve presente sabe o que ocorreu.

### Parâmetros

$$x \in \{0, 1\}^n $$

$$y \in \{0, 1\}$$

Onde $x$ representam as variáveis do acontecimento e $y$ (ou $G(x)$) o seu resultado.

In [ ]:
df = pd.read_csv('titanic.csv')

x = []
y = []

for row in df.iterrows():
  data = row[1]

  group = data.get("Nome")
  age = data.get("Idade")
  gender = data.get("Sexo")
  survived = data.get("Sobreviveu")
  
  if (np.isnan(age) or np.isnan(group) or np.isnan(gender) or np.isnan(survived)):
    continue

  x.append([group, age, gender])
  y.append(int(survived))

## Entradas, pesos, viés, taxa de aprendizado - $n$, $w$, $b$ e $\alpha$

### Quantidade de entradas ($n$)

$$n = |x| $$


A quantidade de entradas é a quantidade de valores $x$ que resultam em um $y$. Nesse caso, para três valores de $x$, um valor de $y$ foi gerado. Logo, o valor de $n$ é a cardinalidade (ou tamanho) do conjunto de $x$ (nesse caso, 3)

In [ ]:
n = len(x[0])

### Pesos ($w$)

$$w = \begin{bmatrix}
a_{1} & ... & a_{n}
\end{bmatrix} \in \mathbb{R}$$

Em um neurônio Adaline, o peso determina a influência de cada entrada e saída do neurônio. De início, o peso é inicializado com valores aleatórios e esse valor é ajustável pela função de aprendizado a medida que o neurônio erra ao tentar reproduzir um valor de  $\hat{y}$ que é diferente de $y$. Como esse é um problema de regressão multivariável, cada peso $w_{i}$ é associado a um valor $x_{i}$ e $w_{i}$ é compartilhável dentre todos os valores $x_{ij}$.

In [ ]:
w = np.random.randn(n)

### Viés ($b$)

$$b = x \in \mathbb{R}$$

O termo de bias é um valor que atua como um compensador e que é aplicado ao final do cálculo da função de inferência. Esse termo independe de qualquer valor $x_{ij}$ e $w_{i}$ e, assim como os pesos, o $b$ é inicializado com um valor aleatório e ajustado na função de aprendizado a medida que o neurônio erra a saída $\hat{y}$ em comparação com $y$.

In [ ]:
b = np.random.rand()

### Taxa de aprendizado ($\alpha$)

$$\alpha = x \in \mathbb{R}$$

A taxa de aprendizado é uma constante real que define o tamanho dos ajustes feitos pela função de aprendizado. Valores maiores representam mudanças mais bruscas na atualização dos pesos ${w_{i}}$ e ${b}$, assim como valores menores representam atualizações mais sutis nos mesmos termos.

In [ ]:
a = 0.0021

## Função de inferência - $\hat{y}$ ou $f(x)$

$$\hat{y} = \sum_{i=1}^{n} w_{i} \cdot x_{i} + b$$

<div align="center">
  ou
</div>

$$\hat{y} = w_{1} \cdot x_{1} + ... + w_{n} \cdot x_{n} + b$$


A função de inferência tenta replicar a função $G(x)$ e reproduzir um valor de $\hat{y}$ baseado em um modelo. Essa tentativa é baseada no somatório da multiplicação entre os pesos $w_{i}$ e os dados $x_{i}$ somado ao termo $b$.

Os parâmetros dessa função ($w$ e $b$) são chamados de modelo $\theta$.

In [ ]:
def inference(x: list, weights: list, bias: float):
  return np.dot(x, weights) + bias

## Função de aprendizado - $F(x)$

$$\varepsilon = y - \hat{y}$$

$$F(x) = \begin{cases}
      w_i^{t+1} &= w_i + \alpha \cdot \varepsilon \cdot x_{i} \\
      b^{t+1} &= b + \alpha \cdot \varepsilon
   \end{cases}$$

A função de aprendizado atualiza os valores do modelo $\theta$ (${w}$ e ${b}$) baseado nos valores de $\alpha$, $\varepsilon$ e $x_{i}$ (no caso de $b$, apenas $\alpha$ e $\varepsilon$). Lembrando que a taxa de erro $\varepsilon$ é calculado pela diferença entre $y$ e $\hat{y}$.

In [ ]:
def learn(x: list, y: float, y_hat: float, weights: list, alpha: float, bias: float):
  error = y - y_hat
  
  for i in range(n):
    weights[i] = weights[i] + alpha * error * x[i]
  bias = bias + alpha * error

  return weights, bias, error

## Função degrau - $d(y)$

$$d: \{\mathbb{R}\} \rightarrow \{0,1\}$$

$$
d(y) = \begin{cases}
      1 & \text{se } y \ge 0 \\
      0 & \text{se } y \lt 1
   \end{cases}
$$

A função de inferência ($f(x)$) retorna um valor real. Porém, para definir se uma pessoa sobreviveu (`True`) ou não (`False`), é necessário adicionar uma função degrau (ou de ativação) para ressignificar o valor de $\hat{y}$

In [ ]:
def step(y: float) -> (int):
  return 0 if y < 1 else 1

## Treinamento

As épocas definem a quantidade de vezes em que o modelo será treinado. Definimos um número de épocas tal que:

$$epoch = x \in \mathbb{N}$$

Nesse caso, vamos definir 5.000 épocas.

Para cada época, vamos tentar um valor $\hat{y}$ por meio da função de inferência $f(x)$. Em seguida, enviamos esse valor $\hat{y}$ juntamente com o valor esperado $y$ e o modelo $\theta$ ($w$ e $b$) para a função de aprendizagem $F(x)$.

A função de aprendizagem utiliza a taxa de erro $\varepsilon = \hat{y} - y$ para atualizar o modelo $\theta$.

In [ ]:
EPOCH = 5000
errors = np.zeros(EPOCH)

for epoch in range(EPOCH):
  for i in range(len(x)):
    random_coordinates = np.random.randint(0, n)

    y_hat = inference(x[random_coordinates], w, b)
    w, b, error = learn(x[random_coordinates], y[random_coordinates], y_hat, w, a, b)
    errors[epoch] += error
    last_error = error

## Resultados do Treinamento

### Gráfico

O gráfico abaixo representa uma relação entre a taxa de erros em cada época.

#

In [ ]:
plt.plot([epoch for epoch in range(EPOCH)], errors)
plt.xlabel('Erros')
plt.ylabel('Época')
plt.title('Relação entre erros e acertos de cada época')

### Relatório

O relatório abaixo mostra os valores do modelo $\theta$ ($w$ e $b$), juntamente com a última taxa de erros $\varepsilon$ registrada. Além disso, todos os valores de $x$ são usados na função de indução $f(x)$ com o modelo $\theta$, dessa forma obtendo os valores para $\hat{y}$.

Em seguida, cada valor de $\hat{y}_{i}$ é comparado com o respectivo valor ${y}_{i}$ para verificar o quão assertivo o treinamento foi.


In [ ]:
y_hat = [step(inference(x[i], w, b)) for i in range(len(x))]
correct = sum(1 for i in range(len(x)) if y[i] == y_hat[i])

print("Results")
print(f"\nTheta\nWeights   = {w}\nBias      = {b}\nError     = {last_error}")
print(f"\nCorrect   = {correct}\nIncorrect = {len(x) - correct}\nTotal     = {len(x)}")

## Testando o Modelo

O trecho de código abaixo testa o modelo treinado acima. Dessa forma, o usuário pode preencher os dados de idade, gênero e classe da seguinte maneira:

Classe = {1,2,3} : A classe do passageiro

Idade = [0, 100]: A idade do passageiros

Sexo = {0,1}: O sexo do passageiro, onde 0 = Masculino e 1 = Feminino

In [68]:
try:
  age = float(input("Digite sua idade: "))
  gender = int(input("Digite o seu gênero (0 ou 1): "))
  group = int(input("Digite o seu grupo (1, 2 ou 3): "))

  survived = step(inference([group, age, gender], w, b))

  print(f"Você {'sobreviveu' if survived else 'não sobreviveu'}")
except ValueError:
  print("O valor é inválido")

Você sobreviveu
